In [6]:
import os

# Check current working directory
print(os.getcwd())

# Change to the root directory of your project, if necessary
os.chdir('C:/Users/yashv/Downloads/HADLN_implementation/HADLN_implementation/')
print(os.getcwd())


C:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation
C:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from datetime import datetime
from preprocessing.lstm_preprocessing import load_data, split_data  # Update this import path as needed
from models.cnn_bilstm_a import create_bi_lstm_model  # Import the LSTM model

In [9]:
import os
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split  # Import for splitting data

# File paths
csv_file_path = 'data/raw/physionet2017.csv'  # Update the path to your CSV file
model_path = 'models/bilstm_A_best.h5'
log_dir = f"results/tensorboard/BiLSTMA/{datetime.now().strftime('%Y%m%d-%H%M%S')}"

# Create directories if they don't exist
os.makedirs(log_dir, exist_ok=True)
os.makedirs('models', exist_ok=True)

# Load and prepare data
# Ensure load_data function is defined properly
ecg_signals, labels_one_hot, num_classes = load_data(csv_file_path)

# Convert one-hot labels back to integer labels for class weights calculation
labels_integer = np.argmax(labels_one_hot, axis=1)

# Split data into training and testing sets (80% train, 20% test split)
X_train, X_test, y_train_one_hot, y_test_one_hot = train_test_split(ecg_signals, labels_one_hot, test_size=0.2, random_state=42)


In [14]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # Adding a feature dimension
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Extract time_step and num_features
time_step = X_train.shape[1]      # Number of time steps
num_features = X_train.shape[2]   # Number of features (should be 1 after reshaping)

# Build and compile the LSTM model
model = create_bi_lstm_model(time_step, num_features, num_classes=num_classes)

# Define a learning rate scheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.5,
                                 patience=3,
                                 min_lr=1e-8)

# Compile the model
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=SigmoidFocalCrossEntropy(),  # Focal Loss for imbalanced classes
              metrics=['accuracy'])


ValueError: Input 0 of layer "bidirectional_5" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 2000)

In [ ]:

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_callback = ModelCheckpoint(filepath=model_path, save_best_only=True, verbose=1)
callbacks = [checkpoint_callback, tensorboard_callback, lr_scheduler]

# Train the model with class weights and callbacks
history = model.fit(X_train, y_train_one_hot,
          validation_data=(X_test, y_test_one_hot),
          epochs=100,
          batch_size=32,
          callbacks=callbacks)

In [ ]:
# Load the trained model
model_path = 'models/cnnbilstma_best.h5'  # Adjust path if necessary
model = tf.keras.models.load_model(model_path)
